### Startup

This code is meant to be executed on Google Colab.
To use it locally change *COLAB_MODE* to False.

**Note**: remember to change *workdir* accordingly, the notebook must be runned inside the root project folder

In [1]:
workdir = "./src"
%cd $workdir

/home/fra/AIGCDetection-CI-CD/src


# Knowledge Distillation

The code incorporates elements derived from the code originally published in the research paper, which can be found here: https://github.com/alsgkals2/CoReD

In [7]:
import sys
from utils.common_functions import *
from utils.cored_functions import *
from utils.data_loader import create_dataloader
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR, OneCycleLR


from utils.data_loader import create_dataloader
from utils.model_loader import load_models
from utils.train_utils import *


def kd_train(args, log = None):

    # Init
    torch.cuda.empty_cache()
    device = 'cuda' if args.num_gpu else 'cpu'
    lr = args.lr
    KD_alpha = args.KD_alpha
    num_class = args.num_class
    num_store_per=5


    # Load datasets and models
    dicLoader, dicCoReD, dicSourceName = initialization(args)
    print("Dataset available in dicLoader: ", " / ".join([n for n in dicLoader]))
    print("Dataset available in dicCoReD: ", " / ".join([n for n in dicCoReD]))
    teacher_model, student_model = load_models(args.weight, args.network, num_gpu = args.num_gpu)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=0.1)

    # Learning rate scheduler
    if args.lr_schedule == "cosine":
        print("Apply Cosine learning rate schedule")
        lr_scheduler = CosineAnnealingLR(optimizer=optimizer,
                                        T_max=10,
                                        eta_min=1e-5,
                                        verbose=True)
    else:
        print(f"Input: {args.lr_schedule}, No learning rate schedule applied ... ")


    # Pre-evaluation
    print("Loading train target for correcting ...  ")
    _list_correct, _ = func_correct(teacher_model.to(device), dicCoReD['train_target_forCorrect'])
    _correct_loaders, already_correct_ratio = GetSplitLoaders_BinaryClasses(_list_correct, dicCoReD['train_target_dataset'], get_augs(args)[0], num_store_per)
    print("Ratio of already correctly predicted in training set: {:.3f}".format(already_correct_ratio))
    list_features = GetListTeacherFeatureFakeReal(teacher_model.module if ',' in args.num_gpu else teacher_model ,_correct_loaders, mode=args.network)
    list_features = np.array(list_features)
    print("List feature size: ", list_features.shape)

    # Initial validation
    _, _, test_acc = Test(dicLoader['val_target'], student_model, criterion, log = log, source_name = args.name_target)
    total_acc = test_acc
    print("[VAL Acc] Target: {:.2f}%".format( test_acc))
    cnt = 1
    for name in dicLoader:
        if 'val_dataset' in name or 'val_source' in name:
            if 'val_source' in name:
                source_name = name.split("_")[2]
            else:
                source_name = "<source name>"

            _, _, source_acc = Test(dicLoader[name], student_model, criterion, log = log, source_name = source_name)
            total_acc += source_acc
            print("[VAL Acc] Source {}-th: {:.2f}%".format(cnt, source_acc))
            cnt += 1

    print("[VAL Acc] Avg {:.2f}%\n Save initial model weight".format(total_acc / cnt))
    best_acc = total_acc
    
    is_best_acc = False
    cur_patience = 0 # Early stop and saving
    l_weight = 1.0 # reduce the conservation when performance does not gain much
    print(f"Start training in {args.epochs} epochs")


    # ------- START TRAINING ------- #
    for epoch in range(args.epochs):
        correct,total = 0,0
        teacher_model.eval()
        student_model.train()
        disp = {}

        for batch_idx, (inputs, targets) in enumerate(dicLoader['train_target']):
            # Load data
            step = (batch_idx+1) * (epoch+1)
            inputs = inputs.to(device).to(torch.float32)
            targets = targets.to(device).to(torch.long)
            if torch.isnan(inputs).any() or torch.isnan(targets).any():
                raise ValueError("There is Nan values in input or target")

            # Forward
            teacher_outputs = teacher_model(inputs)
            penul_ft, outputs = student_model(inputs, True)

            # Losses
            loss_main = criterion(outputs, targets)
            loss_kd = loss_fn_kd(outputs, targets, teacher_outputs)
            loss_kd = loss_clampping(loss_kd, 0, 1800)

            #REP loss
            list_features_std = [list(), list()]
            rep_ft_partitions = correct_binary_simple(inputs=inputs, penul_ft=penul_ft, outputs=outputs, targets=targets) # rep_ft_partitions : 5 x 2
            for j in range(num_store_per):
                for i in range(num_class):
                    if(np.count_nonzero(list_features[i][j])==0 or len(rep_ft_partitions[j][i])==0):
                      continue
                    feat = torch.stack(rep_ft_partitions[j][i], dim=0).mean(dim=0)
                    assert feat.size(-1) == 2048 or feat.size(-1) == 512 or feat.size(-1) == 1280
                    rep_loss = (feat.to(torch.float32)  - torch.tensor(list_features[i][j]).to(device).to(torch.float32)).pow(2).mean()
                    list_features_std[i].append(rep_loss)
            sne_loss = 0.0
            for fs in list_features_std:
                for ss in fs:
                    if ss.requires_grad:
                        sne_loss += ss
            sne_loss = loss_clampping(sne_loss, 0, 1) # REP Loss is clampped in this project

            # Total loss
            loss = loss_main  + l_weight*(loss_kd + sne_loss)
            sne_item = sne_loss if type(sne_loss) == float else sne_loss.item()

            # Log and display
            disp["CE"] = loss_main.item()
            disp["KD"] = loss_kd.item() if loss_kd > 0 else 0.0
            disp["REP"] = sne_item if sne_loss > 0 else 0.0
            call = ' | '.join(["{}: {:.4f}".format(k, v) for k, v in disp.items()])
            print("Train Epoch: {e:03d} Batch: {batch:05d}/{size:05d} | Loss: {loss:.4f} | {call}"
                            .format(e=epoch+1, batch=batch_idx+1, size=len(dicLoader['train_target']), loss=loss.item(), call=call))

            # Learn!
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if args.lr_schedule == "onecycle":
                lr_scheduler.step()

            # Predictions
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == targets).sum().item()
            total += len(targets)

        if args.lr_schedule == "cosine":
            lr_scheduler.step()


        # ----- Validation ------ #

        # Current task
        _, _, test_acc = Test(dicLoader['val_target'], student_model, criterion, log = None, source_name = args.name_target)
        total_acc = test_acc
        print("[VAL Acc] Target: {:.2f}%".format( test_acc))

        # Past tasks
        cnt = 1
        for name in dicLoader:
            if 'val_dataset' in name or 'val_source' in name:
                if 'val_dataset' in name:
                    source_name = dicSourceName[f'source{cnt}']
                else:
                    source_name = dicSourceName['source']

                _, _, source_acc = Test(dicLoader[name], student_model, criterion, log = None, source_name = source_name)
                total_acc += source_acc
                print("[VAL Acc] Source {}-th: {:.2f}%".format(cnt, source_acc))
                cnt += 1
        print("[VAL Acc] Avg {:.2f}%".format(total_acc / cnt))

        # Early stop
        is_best_acc = total_acc > best_acc
        if is_best_acc:
                print("VAL Acc improve from {:.2f}% to {:.2f}%".format(best_acc/cnt, total_acc/cnt))
                cur_patience = 0
        else:
            cur_patience += 1
        if args.loss_schedule and (cur_patience > 0 and cur_patience % 4 == 0):
                l_weight = ReduceWeightOnPlateau(l_weight, args.decay_factor)

        # Save
        best_acc = max(total_acc,best_acc)
        if  is_best_acc:
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': student_model.state_dict(),
                'best_acc': best_acc,
                'optimizer': optimizer.state_dict()},
            checkpoint = savepath,
            filename = 'epoch_{}'.format( epoch+1 if (epoch+1)%10==0 else ''),
            ACC_BEST=is_best_acc
            )
            print('Save best model' if is_best_acc else f'Save checkpoint model @ {epoch+1}')
        if args.early_stop and (cur_patience == args.patience):
            print("Early stopping ...")
            return



# Evaluate


In [5]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import numpy as np
from common_functions import initialization, load_models, AverageMeter
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, average_precision_score
from tqdm import tqdm


def evaluate(args, global_writer=None):

    # Config
    setattr(args,"name_sources", "")
    setattr(args,"name_target", "")



    # Load model
    _, model = load_models(args.weight, args.network, args.num_gpu, not args.test)
    criterion = nn.CrossEntropyLoss().cuda()

    # Load datasets
    tot_avg_acc, real_avg_acc, fake_avg_acc = 0.0, 0.0 ,0.0
    for ds_name in args.ds_cfg["fake_ds"]:
      data_folder = f"{args.dataroot}/{ds_name}/test"
      setattr(args,"data", data_folder)
      dicLoader,_, dicSourceName = initialization(args)


      for key, name in zip(dicLoader, dicSourceName):
        # Init
        global best_acc
        correct, total =0,0
        losses = AverageMeter()
        arc = AverageMeter()
        acc_real = AverageMeter()
        acc_fake = AverageMeter()
        sum_of_AUROC=[]
        target=[]
        output = []
        y_true=np.zeros((0,2),dtype=np.int8)
        y_pred=np.zeros((0,2),dtype=np.int8)

        with torch.no_grad():
          model.eval()
          model.cuda()

          for (inputs, targets) in tqdm(dicLoader[key], ncols=50):
              # Predict
              inputs, targets = inputs.to('cuda'), targets.to('cuda')
              outputs = model(inputs)
              loss = criterion(outputs, targets)
              _, predicted = torch.max(outputs, 1)
              correct = (predicted == targets).squeeze()
              total += len(targets)
              losses.update(loss.data.tolist(), inputs.size(0))
              _y_pred = outputs.cpu().detach()
              _y_gt = targets.cpu().detach().numpy()
              acc = [0, 0]
              class_total = [0, 0]
              for i in range(len(targets)):
                  label = targets[i]
                  acc[label] += 1 if correct[i].item() == True else 0
                  class_total[label] += 1

              losses.update(loss.data.tolist(), inputs.size(0))
              if (class_total[0] != 0):
                  acc_real.update(acc[0] / class_total[0])
              if (class_total[1] != 0):
                  acc_fake.update(acc[1] / class_total[1])

              target.append(_y_gt)
              output.append(_y_pred.numpy()[:,1])
              auroc=None
              try:
                  auroc = roc_auc_score(_y_gt, outputs[:,1].cpu().detach().numpy())
              except ValueError:
                  pass
              sum_of_AUROC.append(auroc)
              _y_true = np.array(torch.zeros(targets.shape[0],2), dtype=np.int8)
              _y_gt = _y_gt.astype(int)
              for _ in range(len(targets)):
                  _y_true[_][_y_gt[_]] = 1
              y_true = np.concatenate((y_true,_y_true))
              a = _y_pred.argmax(1)
              _y_pred = np.array(torch.zeros(_y_pred.shape).scatter(1, a.unsqueeze(1), 1),dtype=np.int8)
              y_pred = np.concatenate((y_pred,_y_pred))

          n_real_samples = np.count_nonzero(y_true, axis=0)[0]
          n_fake_samples = np.count_nonzero(y_true, axis=0)[1]
          acc = accuracy_score(y_true, y_pred)
          ap = average_precision_score(y_true, y_pred)

          result = classification_report(y_true, y_pred,
                                              labels=None,
                                              target_names=None,
                                              sample_weight=None,
                                              digits=4,
                                              output_dict=False,
                                              zero_division='warn')


          print(f"\nLoss:{losses.avg:.4f} | Acc:{acc:.4f} | Acc Real:{acc_real.avg:.4f} | Acc Fake:{acc_fake.avg:.4f} | Ap:{ap:.4f}")
          print(f'Num reals: {n_real_samples}, Num fakes: {n_fake_samples}')
          print("\n\n",result)

          tot_avg_acc += acc
          real_avg_acc += acc_real.avg
          fake_avg_acc += acc_fake.avg

        
    total_ds = len(args.ds_cfg["fake_ds"])
    print(f"Avg: | Acc:{tot_avg_acc/total_ds:.4f} | Acc Real:{real_avg_acc/total_ds:.4f} | Acc Fake:{fake_avg_acc/total_ds:.4f}")


# Workspace

In this section it is possible to run trainings and evaluations

## Traning


1.   Configurate the training
2.   Build the dataset
3.   Train!



### Knowledge distillation

In [3]:
from types import SimpleNamespace

# Set the datasets, it will be used to build the dataset folder
# Available datasets: biggan,crn,cyclegan,faceforensics,gaugan,glow,imle,san,stargan,stylegan,whichfaceisreal,wild",diffusionshort
ds_cfg = {
    "type":         "cddb",                 # cddb, guarnera
    #"real_ds":      "ffhq",                # used only for guarnera: ffhq, celeba
    "fake_ds":      ["cyclegan"] # List of all datasets from first to current task
}

train_cfg = {
    "name":         "tkd_gau_big_cycle",    # Used for tagging the experiment on logs
    "data":         "../datasets",             # Folder containing the EXTRACTED datasets
    "weight":       "../checkpoints/model_best_accuracy.pth", # load weights of task i-1 from file .pth
    "network":      "ResNet",               # Backbone: ResNet, ResNet18, Xception, MobileNet2
    "name_sources": "biggan_cyclegan",        # Ordered list of previous task: dataset1_dataset2_dataseti-1
    "name_target":   "cyclegan",            # Task i dataset
    "checkpoint_path": "../checkpoints/test", # Save folder (the task subfolder is automatically created)
    "lr_schedule":  "cosine",               # cosine, onecycle
    "test":         False,                  # False
    "use_gpu":      True,                   # True, False
    "num_gpu":      "0",                    # GPU id, used only if use_gpu=True
    "loss_schedule": True,                  # True, False
    "num_class":    2,                      # classification classes, 2 for binary classification
    "crop":         True,                   # Crop images instead of resize
    "flip":         False,                  # Random flip augmentation
    "resolution":   128,                    # Crop/resize resolution
    "KD_alpha":     1,                    # alpha factor for kd loss
    "num_store":    5,                      # Stores for representation loss
    "lr":           0.005,                  # Learning rate
    "decay_factor": 0.9,
    "batch_size":   64,                     # Batch size
    "epochs":       5,                    # Traning epochs
    "early_stop":   True,                   # True, False
    "patience":     25,                     # Early stop patience
    "ds_cfg":       ds_cfg,

    "source_datasets":{"cyclegan": "../datasets/cyclegan"},                      
    "target_dataset_name":"biggan",
    "target_dataset_dir":"../datasets/biggan",
    "train":True

}




cfg = SimpleNamespace(**train_cfg)

In [5]:
from train import train

train(cfg)




------ Creating Loaders ------
GPU num is 0

===> Making Loader for Continual Learning..
===> Making Loader : cyclegan
DATASET PATHS
val_source_dir  {'cyclegan': '../datasets/cyclegan'}
val_target_dir  ../datasets/biggan/val/
train_dir  ../datasets/biggan/train/
Dataset available in train_loaders:  train / val
Dataset available in val_loaders:  cyclegan



 ------ Loading models ------
Loading ResNet from ../checkpoints/model_best_accuracy.pth
Loaded
Apply Cosine learning rate schedule
Adjusting learning rate of group 0 to 5.0000e-03.
Start training in 5 epochs
Train Epoch: 001 Batch: 00001/00038 | Loss: 1060.6809 | CE: 0.2185 | KD: 1060.4624
Train Epoch: 001 Batch: 00002/00038 | Loss: 1060.1849 | CE: 0.1700 | KD: 1060.0149
Train Epoch: 001 Batch: 00003/00038 | Loss: 1060.1692 | CE: 0.1514 | KD: 1060.0177
Train Epoch: 001 Batch: 00004/00038 | Loss: 1060.0497 | CE: 0.1415 | KD: 1059.9082
Train Epoch: 001 Batch: 00005/00038 | Loss: 1060.1500 | CE: 0.1805 | KD: 1059.9696
Train Epoch: 0

AttributeError: 'types.SimpleNamespace' object has no attribute 'decay_factor'

In [8]:
kd_train(cfg)




------ Creating Loaders ------
GPU num is 0
Source Name : biggan
Source Name : cyclegan

===> Making Loader for Continual Learning..
===> Making Loader : ../datasets/biggan/val
===> Making Loader : ../datasets/cyclegan/val
DATASET PATHS
val_source_dir  ['../datasets/biggan/val', '../datasets/cyclegan/val']
val_target_dir  ../datasets/cyclegan/val
train_dir  ../datasets/cyclegan/train/
Dataset available in dicLoader:  train_target / val_target / val_dataset1 / val_dataset2
Dataset available in dicCoReD:  train_target_dataset / train_target_forCorrect



 ------ Loading models ------
Loading ResNet from ../checkpoints/model_best_accuracy.pth
Loaded
Apply Cosine learning rate schedule
Adjusting learning rate of group 0 to 5.0000e-03.
Loading train target for correcting ...  


  0%|                      | 0/25 [00:00<?, ?it/s]/home/fra/miniconda3/envs/AIGCdetection/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1706726118919/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
100%|█████████████| 25/25 [00:03<00:00,  7.84it/s]

list_length_realfakeloader : [[23, 26, 26, 51, 615], [15, 26, 56, 125, 520]]
Ratio of already correctly predicted in training set: 0.946


List feature size:  (2, 5, 2048)
===> Starting the dataset cyclegan


100%|██████████| 9/9 [00:00<00:00, 13.43it/s]


...

100%|██████████| 13/13 [00:00<00:00, 14.82it/s]



Test | Loss:0.3038 | MainLoss:0.3038 | top:89.0000
[VAL Acc] Source 1-th: 89.00%
===> Starting the dataset <source name>


100%|██████████| 9/9 [00:00<00:00, 15.53it/s]


Test | Loss:0.1932 | MainLoss:0.1932 | top:91.4122
[VAL Acc] Source 2-th: 91.41%
[VAL Acc] Avg 90.61%
 Save initial model weight
Start training in 5 epochs


NameError: name 'loss_clampping' is not defined

## Evaluation

In [25]:
from types import SimpleNamespace

# Set the datasets, it will be used to build the dataset folder
# Available datasets: biggan,crn,cyclegan,faceforensics,gaugan,glow,imle,san,stargan,stylegan,whichfaceisreal,wild,diffusionshort
ds_cfg = {
    "type":         "cddb",                 # cddb, guarnera
    #"real_ds":      "ffhq",                # used only for guarnera: ffhq, celeba
    "fake_ds":      ["cyclegan"] # List of all datasets to test
}


evaluate_cfg = {

    "name":         "ekd_gau_big_cycle",    # Used for tagging the experiment on logs
    "dataroot":     "../datasets",             # Folder containing the EXTRACTED datasets
    "weight":       "../checkpoints/model_best_accuracy.pth", # load weights of task i from file .pth
    "network":      "ResNet",               # Backbone: ResNet, ResNet18, Xception, MobileNet2
    "test":         True,                   # True
    "use_gpu":      True,                   # True, False
    "num_gpu":      "0",                    # GPU id, used only if use_gpu=True
    "crop":         True,                   # Crop images instead of resize
    "flip":         False,                  # Random flip augmentation
    "resolution":   128,                    # Crop/resize resolution
    "num_class":    2,                      # classification classes, 2 for binary classification
    "batch_size":   64,                     # Batch size
    "ds_cfg":        ds_cfg,
}

evaluate_cfg = SimpleNamespace(**evaluate_cfg)

In [28]:
evaluate(evaluate_cfg)




 ------ Loading models ------
Loading ResNet from ../checkpoints/model_best_accuracy.pth
Loaded



------ Creating Loaders ------
GPU num is 0

===> Starting Task 1 loader from ../datasets/cyclegan/test
Source: 
Target: 


100%|███████████████| 9/9 [00:00<00:00, 11.92it/s]


Loss:0.1773 | Acc:0.9286 | Acc Real:0.9108 | Acc Fake:0.9460 | Ap:0.8983
Num reals: 273, Num fakes: 273


               precision    recall  f1-score   support

           0     0.9466    0.9084    0.9271       273
           1     0.9120    0.9487    0.9300       273

   micro avg     0.9286    0.9286    0.9286       546
   macro avg     0.9293    0.9286    0.9285       546
weighted avg     0.9293    0.9286    0.9285       546
 samples avg     0.9286    0.9286    0.9286       546

Avg: | Acc:0.9286 | Acc Real:0.9108 | Acc Fake:0.9460
